In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(stringr)

lapply(c("dplyr","Seurat","HGNChelper","openxlsx"), library, character.only = T)

load('../data/reference.RData')

sample_meta<-'AD_HS_00001	Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease	20	H.Sapiens	AD	Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex	Braak:0;0;0;2;2;2;2;6;6;6;0;0;0;2;2;2;2;6;6;6	ExcitatoryNeuron;InhibitoryNeuron;Oligodendrocyte;OPC;Astrocyte;Microglia;Endothelia;Pericyte	GSE147528	Single-nucleus RNA-seq	10x Genomics	Male	60;50;71;72;87;91;77;82;72;82;60;50;71;72;87;91;77;82;72;82	SRR11422700;SRR11422701;SRR11422702;SRR11422703;SRR11422704;SRR11422705;SRR11422706;SRR11422707;SRR11422708;SRR11422709;SRR11422710;SRR11422711;SRR11422712;SRR11422713;SRR11422714;SRR11422715;SRR11422716;SRR11422717;SRR11422718;SRR11422719'
split_meta<-unlist(strsplit(sample_meta, '\t'))
samples<-unlist(strsplit(split_meta[14],';'))
disease<-rep(split_meta[5],length(samples))
stage<-unlist(strsplit(unlist(strsplit(split_meta[7],':'))[2],';'))
gender<-rep(split_meta[12],length(samples))
age<-unlist(strsplit(split_meta[13],';'))

sample_meta<-data.frame(samples=samples,disease=disease,stage=stage,gender=gender,age=age)
sample_meta[sample_meta$stage=='0','disease']<-'Control'
rownames(sample_meta)<-sample_meta$samples

# args <- commandArgs(trailingOnly=TRUE)
args<-c('../data/3/scte','../data/3/cell_umap.txt','SRR11422700','/home/wdeng3/workspace/software/sc-type','../../universal_data/rmsk/rmsk_GRCh38.txt')
data_path<-args[1]
out_path<-args[2]
sample_tag<-args[3]
sctype_path<-args[4]
rmsk_path<-args[5]
rmsk<-read.table(rmsk_path,sep='\t',header=F)
to_remove<-unlist(rmsk[!(rmsk$V12 %in% c('LINE','SINE','LTR')),'V11'])
to_remove<-str_replace_all(to_remove,'-','\\.')

reps<-unlist(rmsk[rmsk$V12 %in% c('LINE','SINE','LTR'),'V11'])
reps<-str_replace_all(reps,'-','\\.')

create_seurat<-function(sample_tag){
    ## load data
    scte.data <- t(read.csv(paste0(data_path,'/',sample_tag,'/',sample_tag,'.csv'),check.names=F, row.names = 1))
    # remove repeats from gene expression matrix

    scte.gene<-scte.data[!(row.names(scte.data) %in% to_remove),]
    # normalize and scale data
    scte <- CreateSeuratObject(counts = scte.gene, project = sample_tag, min.cells = 3, min.features = 200)
    # normalize data
    scte[["percent.mt"]] <- PercentageFeatureSet(scte, pattern = "^MT.|^MT-")
    scte <- subset(scte, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
    scte <- SCTransform(scte, vars.to.regress = "percent.mt", verbose = FALSE)

    cell_meta<-scte@meta.data
    cell_meta$disease<-sample_meta[cell_meta$orig.ident,'disease']

    cell_meta$stage<-sample_meta[cell_meta$orig.ident,'stage']
    cell_meta$gender<-sample_meta[cell_meta$orig.ident,'gender']
    cell_meta$age<-sample_meta[cell_meta$orig.ident,'age']
    scte@meta.data<-cell_meta
    scte
}

scte<-create_seurat(sample_tag)

scte <- FindVariableFeatures(scte, selection.method = "vst", nfeatures = 2000)
# scale and run PCA
scte <- ScaleData(scte, features = rownames(scte))
scte <- RunPCA(scte, features = VariableFeatures(object = scte))

sdv<-Stdev(scte,reduction='pca')
sdv<-sdv[sdv>2]
npca<-length(sdv)

scte <- FindNeighbors(scte, dims = 1:npca)
scte <- FindClusters(scte, resolution = 0.8)
scte <- RunUMAP(scte, dims = 1:npca)


scte.anchors <- FindTransferAnchors(reference = reference, query = scte,
    dims = 1:30, reference.reduction = "pca")
scte.query <- MapQuery(anchorset = scte.anchors, reference = reference, query = scte,
    refdata = list(celltype = "broad.cell.type"), reference.reduction = "pca", reduction.model = "umap")
cell_meta<-scte.query@meta.data

cell_meta$UMAP_1<-scte.query@reductions[['umap']]@cell.embeddings[,1]
cell_meta$UMAP_2<-scte.query@reductions[['umap']]@cell.embeddings[,2]
scte.query@meta.data<-cell_meta
write.table(cell_meta,out_path,sep='\t',quote=F,row.names=T)

## get gene expression from scte_query
scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)

In [9]:
import os
import sys

out_dir='/Volumes/T7/scRTE/splitted/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for mysql in os.listdir('/Volumes/T7/scRTE/'):
    if mysql.startswith('cell_exp_'):
        print(mysql)
        out_file=open(out_dir+mysql,'w')
        for line in open('/Volumes/T7/scRTE/'+mysql):
            if not line.startswith('INSERT INTO'):
                out_file.write(line)
            else:
                info=line.split('values ')
                info[1]=info[1].strip()[1:-2].split('),(')
                for i in range(8):
                    out_file.write(info[0]+'values ')
                    out_file.write('({content});\n'.format(content='),('.join(info[1][i*250:(i+1)*250])))

cell_exp_0.sql
cell_exp_39.sql
cell_exp_38.sql
cell_exp_37.sql
cell_exp_36.sql
cell_exp_35.sql
cell_exp_34.sql
cell_exp_33.sql
cell_exp_32.sql
cell_exp_31.sql
cell_exp_30.sql
cell_exp_29.sql
cell_exp_28.sql
cell_exp_27.sql
cell_exp_26.sql
cell_exp_25.sql
cell_exp_23.sql
cell_exp_22.sql
cell_exp_21.sql
cell_exp_20.sql
cell_exp_19.sql
cell_exp_18.sql
cell_exp_17.sql
cell_exp_16.sql
cell_exp_15.sql
cell_exp_14.sql
cell_exp_13.sql
cell_exp_12.sql
cell_exp_11.sql
cell_exp_10.sql
cell_exp_9.sql
cell_exp_8.sql
cell_exp_7.sql
cell_exp_6.sql
cell_exp_5.sql
cell_exp_4.sql
cell_exp_3.sql
cell_exp_2.sql
cell_exp_1.sql
cell_exp_24.sql
cell_exp_44.sql
cell_exp_43.sql
cell_exp_42.sql
cell_exp_41.sql
cell_exp_40.sql


In [1]:
import subprocess
to_compress=[]
for mysql in os.listdir('/Volumes/T7/scRTE/splitted'):
    if mysql.startswith('cell_exp_'):
        to_compress.append('/Volumes/T7/scRTE/splitted/'+mysql)

subprocess.call('tar -zcvf /Volumes/T7/scRTE/cell_exp.tar.gz '+' '.join(to_compress),shell=True)

tar: Removing leading '/' from member names
a Volumes/T7/scRTE/splitted/cell_exp_0.sql

KeyboardInterrupt: 

In [ ]:
# ## assign cell type according to ScType tuturial

## load sc type and database
# source(paste0(sctype_path,"/R/gene_sets_prepare.R"))
# source(paste0(sctype_path,"/R/sctype_score_.R"))
# gs_list = gene_sets_prepare(paste0(sctype_path,'/ScTypeDB_full.xlsx'), "Brain" )
# # get cell-type by cell matrix
# es.max = sctype_score(scRNAseqData = scte[["RNA"]]@scale.data, scaled = TRUE, 
#                       gs = gs_list$gs_positive, gs2 = gs_list$gs_negative) 

# # merge by cluster
# cL_resutls = do.call("rbind", lapply(unique(scte@meta.data$seurat_clusters), function(cl){
#     es.max.cl = sort(rowSums(es.max[ ,rownames(scte@meta.data[scte@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
#     head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(scte@meta.data$seurat_clusters==cl)), 10)
# }))
# sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# # set low-confident (low ScType score) clusters to "unknown"
# sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"


# scte@meta.data$customclassif = ""
# for(j in unique(sctype_scores$cluster)){
#   cl_type = sctype_scores[sctype_scores$cluster==j,]; 
#   scte@meta.data$customclassif[scte@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
# }

# ## save cell type
# cell_types<-scte[['customclassif']]
